In [ ]:
'''
baseline version that is basically without using a graph
see how much can be pushed to the LLM

working
prompt
    use some of your old prompts and questions

to do
prompt
simple graph
gradio interface
    look over old notes
evaluate
    better testing questions




backlog
    see other notes for that


'''

### Constants

In [ ]:
import gradio as gr
import getpass
import os

from IPython.display import Image, display

from langchain_openai import OpenAI, ChatOpenAI

from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import Runnable, RunnableConfig

from pydantic import BaseModel, Field

from typing import Annotated, Literal, Optional

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages

from langchain_core.tools import tool

from typing import Literal

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import tools_condition

import pgeocode

import uuid

In [ ]:
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGSMITH_API_KEY")
# _set_env("TAVILY_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "EV Prototype v0.4"


In [ ]:
testing_questions_list = [
    'What is the cost of owning a 2024 new Hyundai Ioniq versus a 2024 Hyundai Venue?',

]

In [ ]:
def display_graph(graph):
    try:
        display(Image(graph.get_graph().draw_mermaid_png()))
    except Exception:
        # This requires some extra dependencies and is optional
        pass


def _print_event(event: dict, _printed: set, max_length=1500):
    current_state = event.get("dialog_state")
    if current_state:
        print("Currently in: ", current_state[-1])
    message = event.get("messages")
    if message:
        if isinstance(message, list):
            message = message[-1]
        if message.id not in _printed:
            msg_repr = message.pretty_repr(html=True)
            if len(msg_repr) > max_length:
                msg_repr = msg_repr[:max_length] + " ... (truncated)"
            print(msg_repr)
            _printed.add(message.id)

### v 0.01 Minimal LLM based

In [ ]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    user_info: str

In [ ]:
class Assistant:
    def __init__(self, runnable: Runnable):
        self.runnable = runnable

    def __call__(self, state: State, config: RunnableConfig):
        while True:
            result = self.runnable.invoke(state)

            if not result.tool_calls and (
                not result.content
                or isinstance(result.content, list)
                and not result.content[0].get("text")
            ):
                messages = state["messages"] + [("user", "Respond with a real output.")]
                state = {**state, "messages": messages}
            else:
                break
        return {"messages": result}


llm = ChatOpenAI(model_name="gpt-3.5-turbo")


ev_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant that helps gather information for people deciding to buy or lease an electric vehicle or a gas car."
            "Help them compare the costs of buying or leasing an electric vehicle or a gas car."
            "The fundamental question you are trying to answer is whether it is cheaper to buy an electric vehicle or a gas car."
            "If the user does not provide detailed information, answer the question generally."
            "If the user does provide detailed information, use that information to provider a better answer."
            "Try to ask the user for more information after you provider your answer."
            "Some examples of key pieces of information to ask include:\n"
            " - Location\n"
            " - Used or new\n"
            " - Type of car they are interested in (sedan, luxury, compact-SUV, mid-size truck, etc.) \n"
        ),
        ("placeholder", "{messages}"),
    ]
)

ev_assistant_runnable = ev_prompt | llm

builder = StateGraph(State)

builder.add_node("assistant", Assistant(ev_assistant_runnable))
# builder.set_entry_point("assistant")
builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

memory = MemorySaver()
baseline_agent_graph = builder.compile(
    checkpoint=memory,
)

display_graph(baseline_agent_graph)

In [ ]:
thread_id = str(uuid.uuid4())

config = {
    "configurable": {

        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

_printed = set()
for question in testing_questions_list:
    events = baseline_agent_graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)
